In [ ]:
import os
import sys
import traceback

import main_fte
from py_utils import data_ops
import evaluate_fte
from lib import app
from py_utils import log

%load_ext autoreload
%autoreload 2

%matplotlib inline

# root_dir = os.path.join("..", "data")
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")
logger = log.logger("main")

In [ ]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = main_fte.create_pose_functions(root_dir)

In [ ]:
# Configurations for the optimisation.
# Test files: 2017_08_29/top/jules/run1_1 (10, 110, 1), 2017_09_02/top/phantom/run1_3 (51, 130, 4)
# 2017_09_02/top/jules/run1 (22, 100, 1), 2017_08_29/top/phantom/run1_1 (20, 170, 1)
data_path = os.path.join("2017_08_29", "top", "jules", "run1_1")
start_frame, end_frame, cam_idx = (10, 110, 1)
dlc_thresh = 0.5

In [ ]:
# Run the optimisation
try:
    main_fte.run(root_dir, data_path, cam_idx, start_frame, end_frame, dlc_thresh, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")
except:
    logger.error(traceback.format_exc())
    # Stop logging and return print functionality to normal.
    sys.stdout.logfile.close()
    sys.stdout = sys.stdout.terminal

In [ ]:
dummy_scene = "/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos/2017_08_29/top/extrinsic_calib/6_cam_scene_sba.json"
main_fte.plot_trajectory("/Users/zico/msc/dev/monocular_estimation/data/synthesized.pickle", dummy_scene)

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.plot_cheetah(root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.compare_cheetahs("/Users/zico/msc/dev/AcinoSet/data/2017_08_29/top/jules/run1_1/fte_1/fte.pickle", root_dir, data_path, fte_type="sd_fte", centered=False, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
main_fte.compare_traj_error("/Users/zico/msc/dev/AcinoSet/data/2017_08_29/top/jules/run1_1/fte_pw_2/fte.pickle", "/Users/zico/msc/dev/AcinoSet/data/2017_08_29/top/jules/run1_1/fte_1/fte.pickle", root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
eval_dir = os.path.join("/Users/zico/msc/dev/AcinoSet/data", data_path, "fte_pw")
data = data_ops.load_pickle(os.path.join(eval_dir, "fte.pickle"))
results_dir = os.path.join(eval_dir, "evaluation")
os.makedirs(eval_dir, exist_ok=True)
max_delta_acc = evaluate_fte.eval_delta_acc(data, results_dir)
evaluate_fte.eval_model_error(data, results_dir)
evaluate_fte.eval_meas_error(data, results_dir)
logger.info(max_delta_acc)

In [ ]:
fig_fpath = os.path.join(os.path.join(os.path.join("..", "data"), data_path, "fte_pw"), 'fte.svg')
pw_file = data_ops.load_pickle(os.path.join(os.path.join("..", "data"), data_path, "fte_pw", "fte.pickle"))
app.plot_cheetah_states(pw_file["x"], out_fpath=fig_fpath)

In [ ]:
main_fte.plot_cost_functions()